In [1]:
import sys
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
from pprint import pprint
from importlib import reload

%matplotlib notebook

In [2]:
paths = [r"c:\Users\Administrator\Code\pulsar2"]

for p in paths:
    if p not in sys.path:
        sys.path.append(p)

In [3]:
%run ../Experiments/init_vars.py
%run ../Experiments/init_instruments.py

station = qcodes.Station(key, ivvi, RF, LO, alazar, fg, awg)

2017-12-30 18:24:23,840 : measurement : INFO : Logger set up!


Initialized IVVI-rack in 0.08s


C:\Users\Administrator\Code\Qcodes\qcodes\instrument\parameter.py:209: UserWarning: Wrapping get method, original get method will not be directly accessible. It is recommended to define get_raw in your subclass instead.
  warnings.warn('Wrapping get method, original get method will not '


Connected to: Keysight Technologies 34465A (serial:MY57503596, firmware:A.02.14-02.40-02.14-00.49-03-01) in 0.70s
Connected to: Rohde&Schwarz SGS100A (serial:1416.0505k02/110525, firmware:3.1.19.15-3.50.124.67) in 2.03s
Connected to: Rohde&Schwarz SGS100A (serial:1416.0505k02/110524, firmware:3.1.19.15-3.50.124.67) in 1.96s
Connected to: TEKTRONIX AWG5014C (serial:B051388, firmware:SCPI:99.0 FW:4.6.0.7) in 1.83s
Connected to: Rigol Technologies DG4062 (serial:DG4E163251554, firmware:00.01.09) in 0.05s


C:\Users\Administrator\Code\Qcodes\qcodes\instrument\parameter.py:514: UserWarning: set_step is deprecated use step property as in `inst.step = stepvalue` instead
  "set_step is deprecated use step property as in `inst.step = "
C:\Users\Administrator\Code\Qcodes\qcodes\instrument\parameter.py:526: UserWarning: set_delay is deprecated use inter_delay or post_delay property as in `inst.inter_delay = delayvalue` instead
  "set_delay is deprecated use inter_delay or post_delay property "


In [8]:
class SimpleRamp(AWGMeasurement):
    
    def __init__(self, *arg, **kw):
        super().__init__(*arg, **kw)
        
        self.add_parameter('npts', Parameter, initial_value=11)
        self.add_parameter('max_val', Parameter, initial_value=0.1)
        self.add_parameter('val_len', Parameter, initial_value=1e-7)
        self.add_parameter('ramp_down_len', Parameter, initial_value=1e-6)
        self.add_parameter('awg_chan', Parameter, initial_value=2)

    def awg_sequence(self):
        h = self.awghandler
        chan = self.awg_chan()
        
        samples_per_val = h.time_to_samples(self.val_len())
        samples_ramp_down = h.time_to_samples(self.ramp_down_len())
        vals = np.linspace(0, h.voltage_to_wfscale()[self.awg_chan()] * self.max_val(), self.npts())
        
        wf = np.zeros((self.npts()*samples_per_val+samples_ramp_down,), 
                      dtype=h.get_awg_wf_dtype(chans=[chan]))  
        
        for i, v in enumerate(vals):
            idx0, idx1 = i*samples_per_val, (i+1)*samples_per_val
            wf[f'ch{chan}_wf'][idx0:idx1] = v
        wf[f'ch{chan}_wf'][idx1:] = np.linspace(v, 0, samples_ramp_down)
        
        return [dict(wf=wf)]


m = SimpleRamp(station, namespace)
m.namespace.awg_settings['sampling_rate'] = 1e7
m.namespace.awg_settings['channel_{}'.format(m.awg_chan())]['analog_amplitude'] = 2*m.max_val()
m.setup()